Importer la table **TRANSACTIONS**

In [1]:
import pandas as pd

df_transactions = pd.read_csv("transactions.csv", parse_dates=["created_date"])
df_transactions.head()

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,ea_cardholderpresence,ea_merchant_mcc,ea_merchant_city,ea_merchant_country,direction,user_id,created_date,merchant_country,outlier_level
0,transaction_505,topup,aed,298.62,completed,Not Applicable,-1.0,Not Applicable,Not Applicable,inbound,user_4773,2018-08-02 09:41:07,Not Applicable,extreme
1,transaction_1755379,exchange,aed,3.51,completed,Not Applicable,-1.0,Not Applicable,Not Applicable,outbound,user_10435,2018-09-06 10:23:15,Not Applicable,normal
2,transaction_1667,exchange,aed,0.05,completed,Not Applicable,-1.0,Not Applicable,Not Applicable,outbound,user_397,2018-05-16 06:09:28,Not Applicable,normal
3,transaction_1429,exchange,aed,24.14,completed,Not Applicable,-1.0,Not Applicable,Not Applicable,outbound,user_4399,2018-12-19 20:22:03,Not Applicable,normal
4,transaction_345,exchange,aed,0.21,completed,Not Applicable,-1.0,Not Applicable,Not Applicable,outbound,user_1731,2018-10-27 13:29:56,Not Applicable,normal


In [2]:
df_transactions.shape

(2740075, 14)

In [3]:
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2740075 entries, 0 to 2740074
Data columns (total 14 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   transaction_id         object        
 1   transactions_type      object        
 2   transactions_currency  object        
 3   amount_usd             float64       
 4   transactions_state     object        
 5   ea_cardholderpresence  object        
 6   ea_merchant_mcc        float64       
 7   ea_merchant_city       object        
 8   ea_merchant_country    object        
 9   direction              object        
 10  user_id                object        
 11  created_date           datetime64[ns]
 12  merchant_country       object        
 13  outlier_level          object        
dtypes: datetime64[ns](1), float64(2), object(11)
memory usage: 292.7+ MB


In [4]:
df_transactions["user_id"].nunique()

18766

**Fixer le reference date**

In [5]:
reference_date = df_transactions['created_date'].max()

In [6]:
reference_date

Timestamp('2019-05-16 18:22:16')

**Garder uniquement les COMPLETED**

In [7]:
completed_tx = df_transactions[df_transactions['transactions_state'] == 'completed'].copy()

**Calculer les metrics (completed_metrics = uniquement les users qui ont ≥1 completed tx)**

In [8]:
completed_metrics = completed_tx.groupby('user_id').agg(
    first_completed_tx=('created_date','min'),
    last_completed_tx=('created_date','max'),
    tx_completed_count=('transaction_id','nunique'),
    tx_amount_total=('amount_usd','sum'),
    tx_amount_avg=('amount_usd','mean')
).reset_index()

**Calculer la RECENCY**

In [9]:
completed_metrics['days_since_last_completed'] = (reference_date - completed_metrics['last_completed_tx']).dt.days

**Créer le churn flag (transaction-level)**

In [10]:
completed_metrics['churn_flag'] = (completed_metrics['days_since_last_completed'] > 30).astype(int)

Pour l instant le churn flag ne contient que les users ayant eu au moins une COMPLETED transation.

**Importer la table USERS**

In [11]:
df_users = pd.read_csv("users.csv", parse_dates=["created_date"])
df_users.head()

,user_id,birth_year,country,city,created_date,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,num_referrals,num_successful_referrals,city_clean
0,user_2811,1988,IE,C,2018-03-04 21:46:39.616834+00:00,0,STANDARD,Not Informed,Not Informed,0,0,0,Unknown
1,user_4750,1987,FR,Ry,2018-04-07 06:49:18.345736+00:00,0,STANDARD,Not Informed,Not Informed,0,0,0,Unknown
2,user_17686,1984,GB,Ayr,2018-12-04 10:28:02.653147+00:00,0,PREMIUM,Not Informed,Not Informed,6,0,0,Ayr
3,user_18779,1965,GB,Ayr,2018-12-31 07:42:19.353613+00:00,0,STANDARD,Not Informed,Not Informed,0,0,0,Ayr
4,user_7823,1999,PL,Buk,2018-06-07 03:22:40.590209+00:00,0,STANDARD,Not Informed,Not Informed,0,0,0,Buk


In [12]:
df_users.shape

(19430, 13)

In [13]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19430 entries, 0 to 19429
Data columns (total 13 columns):
 #   Column                                    Non-Null Count  Dtype              
---  ------                                    --------------  -----              
 0   user_id                                   19430 non-null  object             
 1   birth_year                                19430 non-null  int64              
 2   country                                   19430 non-null  object             
 3   city                                      19430 non-null  object             
 4   created_date                              19430 non-null  datetime64[ns, UTC]
 5   user_settings_crypto_unlocked             19430 non-null  int64              
 6   plan                                      19430 non-null  object             
 7   attributes_notifications_marketing_push   19430 non-null  object             
 8   attributes_notifications_marketing_email  19430 non-null

In [14]:
df_users["user_id"].nunique()

19430

**Ajouter les users SANS COMPLETED transactions** ( merge avec colonne User_id de df_users)

In [15]:
user_activity = df_users[['user_id']].merge(
    completed_metrics,
    on='user_id',
    how='left'
)

**Gérer les users sans activité (Users sans completed transactions = churned)**

In [16]:
user_activity['churn_flag'] = user_activity['churn_flag'].fillna(1).astype(int)

In [17]:
user_activity.head()

,user_id,first_completed_tx,last_completed_tx,tx_completed_count,tx_amount_total,tx_amount_avg,days_since_last_completed,churn_flag
0,user_2811,2018-03-21 23:36:16,2018-05-20 07:45:36,57.0,9341.32,163.882807,361.0,1
1,user_4750,2018-04-07 06:53:12,2018-11-04 07:41:39,79.0,2343.19,29.660633,193.0,1
2,user_17686,2018-12-04 10:30:42,2019-05-11 09:33:37,174.0,8562.77,49.211322,5.0,0
3,user_18779,2018-12-31 07:46:58,2018-12-31 07:46:58,1.0,10.00,10.000000,136.0,1
4,user_7823,2018-06-07 03:37:03,2019-02-04 05:57:30,21.0,297.55,14.169048,101.0,1


remplir le recency avec "9999" pour eviter les problemes de NAN en power BI et filtrer plus facilement

In [18]:
user_activity['days_since_last_completed'] = user_activity['days_since_last_completed'].fillna(9999).astype(int)

**Vérification finale**

In [19]:
user_activity.head()

,user_id,first_completed_tx,last_completed_tx,tx_completed_count,tx_amount_total,tx_amount_avg,days_since_last_completed,churn_flag
0,user_2811,2018-03-21 23:36:16,2018-05-20 07:45:36,57.0,9341.32,163.882807,361,1
1,user_4750,2018-04-07 06:53:12,2018-11-04 07:41:39,79.0,2343.19,29.660633,193,1
2,user_17686,2018-12-04 10:30:42,2019-05-11 09:33:37,174.0,8562.77,49.211322,5,0
3,user_18779,2018-12-31 07:46:58,2018-12-31 07:46:58,1.0,10.00,10.000000,136,1
4,user_7823,2018-06-07 03:37:03,2019-02-04 05:57:30,21.0,297.55,14.169048,101,1


In [20]:
user_activity.shape

(19430, 8)

In [21]:
user_activity["user_id"].nunique()

19430

In [22]:
user_activity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19430 entries, 0 to 19429
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   user_id                    19430 non-null  object        
 1   first_completed_tx         18529 non-null  datetime64[ns]
 2   last_completed_tx          18529 non-null  datetime64[ns]
 3   tx_completed_count         18529 non-null  float64       
 4   tx_amount_total            18529 non-null  float64       
 5   tx_amount_avg              18529 non-null  float64       
 6   days_since_last_completed  19430 non-null  int64         
 7   churn_flag                 19430 non-null  int64         
dtypes: datetime64[ns](2), float64(3), int64(2), object(1)
memory usage: 1.2+ MB


Cohérence churn

In [23]:
user_activity['churn_flag'].value_counts()

,count
churn_flag,
0,10696
1,8734


Logique correcte

Si Churn = 0 , alors max 'days_since_last_completed' doit etre inferieur ou egal a 30

In [24]:
user_activity[user_activity['churn_flag']==0]['days_since_last_completed'].max()

30

Si Churn = 1 , alors min 'days_since_last_completed' doit etre superieur a 30

In [25]:
user_activity[user_activity['churn_flag']==1]['days_since_last_completed'].min()

31

Users sans activité

users sans activité j avais remplacé NAN par 9999

In [26]:
(user_activity['days_since_last_completed'] == 9999).sum()

np.int64(901)